In [40]:
# Import Libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook

In [41]:
# Load API Credentials

with open('/Users/kritpatel/.secret/yelp_api.json') as f:
    creds = json.load(f)

In [42]:
# Check if the Credentials are loaded

creds.keys()

dict_keys(['Client ID', 'API Key'])

In [43]:
# Define YelpAPI Variable

yelp = YelpAPI(creds['API Key'], timeout_s= 5.0)
yelp

In [44]:
# Set the API cell parameters

LOCATION = 'Chicago, IL 60606'
TERM = 'Pizza'

In [45]:
# Folder for Saving the Data

FOLDER = 'Data/'
os.makedirs(FOLDER, exist_ok = True)

In [46]:
# Specifying the JSON_FILE

JSON_FILE = FOLDER + 'Chicago Pizza.json'

In [47]:
# Perform our API call

results = yelp.search_query(term= TERM, location= LOCATION)
type(results)

dict

In [48]:
# Check the results

results.keys()

dict_keys(['businesses', 'total', 'region'])

In [49]:
# Check for Businesses

results['businesses']

[{'id': '8vFJH_paXsMocmEO_KAa3w',
  'alias': 'lou-malnatis-pizzeria-chicago',
  'name': "Lou Malnati's Pizzeria",
  'image_url': 'https://s3-media3.fl.yelpcdn.com/bphoto/9FiL-9Pbytyg6usOE02lYg/o.jpg',
  'is_closed': False,
  'url': 'https://www.yelp.com/biz/lou-malnatis-pizzeria-chicago?adjust_creative=nlmqn02SsEKFg8L96PXwWw&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=nlmqn02SsEKFg8L96PXwWw',
  'review_count': 6654,
  'categories': [{'alias': 'pizza', 'title': 'Pizza'},
   {'alias': 'italian', 'title': 'Italian'},
   {'alias': 'sandwiches', 'title': 'Sandwiches'}],
  'rating': 4.0,
  'coordinates': {'latitude': 41.890357, 'longitude': -87.633704},
  'transactions': ['pickup', 'delivery'],
  'price': '$$',
  'location': {'address1': '439 N Wells St',
   'address2': '',
   'address3': '',
   'city': 'Chicago',
   'zip_code': '60654',
   'country': 'US',
   'state': 'IL',
   'display_address': ['439 N Wells St', 'Chicago, IL 60654']},
  'phone': '+13128289800',
 

In [50]:
# Check for Region

results['region']

{'center': {'longitude': -87.63751029968262, 'latitude': 41.88291699238067}}

In [51]:
# Check for Total Results

total_results = results['total']
total_results

626

In [52]:
# Check for results per page

results_per_page = len(results['businesses'])
results_per_page

20

In [53]:
# Check for Total number of pages

n_pages = math.ceil(total_results / results_per_page)
n_pages

32

In [54]:
# Save the results in the file

with open(JSON_FILE, 'w') as f:
    json.dump(results['businesses'], f)

In [55]:
for i in tqdm_notebook(range(1, n_pages + 1)):
    
    # Read the results in progress file and check the length
    
    with open(JSON_FILE) as f:
        prev_results = json.load(f)
        
        
    # Set the offset based on previous results
    
    n_results = len(prev_results)
    
    
    # run another call with offet equal to n_results
    
    results = yelp.search_query(term= TERM, location= LOCATION,
                                offset= n_results, limit= 5)
    
    
    # Append New results and Save to File
    
    prev_results.extend(results['businesses'])
    with open(JSON_FILE,'w') as f:
        json.dump(prev_results,f)
    

  0%|          | 0/32 [00:00<?, ?it/s]

In [56]:
# Show final results

df = pd.read_json(JSON_FILE)
display(df.head(), df.tail())

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,8vFJH_paXsMocmEO_KAa3w,lou-malnatis-pizzeria-chicago,Lou Malnati's Pizzeria,https://s3-media3.fl.yelpcdn.com/bphoto/9FiL-9...,False,https://www.yelp.com/biz/lou-malnatis-pizzeria...,6654,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.0,"{'latitude': 41.890357, 'longitude': -87.633704}","[pickup, delivery]",$$,"{'address1': '439 N Wells St', 'address2': '',...",+13128289800,(312) 828-9800,885.262704
1,Bxd9Y2PsdvIOGIKFQh53RQ,giordanos-chicago-30,Giordano's,https://s3-media1.fl.yelpcdn.com/bphoto/OKxuR9...,False,https://www.yelp.com/biz/giordanos-chicago-30?...,3175,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.0,"{'latitude': 41.88514, 'longitude': -87.62313}",[delivery],$$,"{'address1': '130 E Randolph', 'address2': '',...",+13126161200,(312) 616-1200,1199.724383
2,8eEOnlfXZFYhyzr52ugvCg,parlor-pizza-bar-river-north-chicago,Parlor Pizza Bar - River North,https://s3-media2.fl.yelpcdn.com/bphoto/naQHnM...,False,https://www.yelp.com/biz/parlor-pizza-bar-rive...,277,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.0,"{'latitude': 41.8894378, 'longitude': -87.6294...","[pickup, delivery]",$$,"{'address1': '405 N Dearborn St', 'address2': ...",+13123922356,(312) 392-2356,995.480690
3,LNtuCsJLk3E9y7I5sd5SjQ,giordanos-jackson-chicago,Giordano's - Jackson,https://s3-media1.fl.yelpcdn.com/bphoto/LcX7fa...,False,https://www.yelp.com/biz/giordanos-jackson-chi...,1315,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.0,"{'latitude': 41.8779751, 'longitude': -87.6344...","[pickup, delivery]",$$,"{'address1': '223 W Jackson Blvd', 'address2':...",+13125839400,(312) 583-9400,599.929386
4,4f0lfn_AnA38aUtvW81tcw,vinnys-pizza-bar-chicago,Vinny's Pizza Bar,https://s3-media2.fl.yelpcdn.com/bphoto/hVoMkX...,False,https://www.yelp.com/biz/vinnys-pizza-bar-chic...,23,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.0,"{'latitude': 41.8842375, 'longitude': -87.6401...","[pickup, delivery]",NaN,"{'address1': '130 N Canal St', 'address2': Non...",+13129021544,(312) 902-1544,263.660912


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
175,p0UquNGFSGewziWPevA0Ng,dylans-tavern-and-grill-chicago,Dylans Tavern & Grill,https://s3-media3.fl.yelpcdn.com/bphoto/Ujbyrg...,False,https://www.yelp.com/biz/dylans-tavern-and-gri...,252,"[{'alias': 'newamerican', 'title': 'American (...",3.0,"{'latitude': 41.8799, 'longitude': -87.64144}","[delivery, pickup]",$$,"{'address1': '118 S Clinton St', 'address2': '...",+13128762008,(312) 876-2008,462.437171
176,AJw1pkVQJ-DmXU89CgZ9-w,shamrock-club-chicago,Shamrock Club,https://s3-media1.fl.yelpcdn.com/bphoto/WusMWB...,False,https://www.yelp.com/biz/shamrock-club-chicago...,166,"[{'alias': 'newamerican', 'title': 'American (...",4.0,"{'latitude': 41.8893394730647, 'longitude': -8...","[delivery, pickup]",$,"{'address1': '210 W Kinzie St', 'address2': ''...",+13123219314,(312) 321-9314,752.471800
177,U_OhWaDWwym5garJ5KyYMA,safehouse-chicago-chicago,SafeHouse Chicago,https://s3-media1.fl.yelpcdn.com/bphoto/jMUxnB...,False,https://www.yelp.com/biz/safehouse-chicago-chi...,1345,"[{'alias': 'burgers', 'title': 'Burgers'}, {'a...",4.0,"{'latitude': 41.8934761420147, 'longitude': -8...",[delivery],$$,"{'address1': '60 E Ontario St', 'address2': No...",+13123131007,(312) 313-1007,1513.829452
178,C0xde8M49hJ3dtKuUTgflg,flight-club-darts-chicago-chicago,Flight Club Darts Chicago,https://s3-media1.fl.yelpcdn.com/bphoto/fOoP41...,False,https://www.yelp.com/biz/flight-club-darts-chi...,224,"[{'alias': 'cocktailbars', 'title': 'Cocktail ...",4.5,"{'latitude': 41.8867465458374, 'longitude': -8...",[delivery],$$,"{'address1': '111 W Wacker Dr', 'address2': No...",+13122842474,(312) 284-2474,671.946917
179,Sfx5Yb95Wav1aOa0Qkv1VA,mercadito-chicago,Mercadito,https://s3-media4.fl.yelpcdn.com/bphoto/jE5s7j...,False,https://www.yelp.com/biz/mercadito-chicago?adj...,2288,"[{'alias': 'mexican', 'title': 'Mexican'}, {'a...",3.5,"{'latitude': 41.8894499, 'longitude': -87.63162}","[delivery, pickup]",$$,"{'address1': '108 W Kinzie St', 'address2': ''...",+13123299555,(312) 329-9555,874.108323


In [57]:
# Check for duplicated valuesx

df.duplicated(subset='id').sum()

3

In [58]:
# Drop Duplicated values

df.drop_duplicates(subset='id', inplace= True)
df.duplicated(subset='id').sum()

0

In [59]:
# Save the final results to a compressed csv

df.to_csv('Data/final_results_Chicago_Pizza.csv.gz', compression='gzip',index=False)